In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Baseline SVD using Surprise
### RecommendatIon System Engine

In [2]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

### inspecting the data

### Reading in the dataset and prepping data

In [3]:
df_review = pd.read_csv('Yelp_Dataset_Reviews_new.csv')
# .drop(['Unnamed: 0'], axis=1)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# for Surprise, we just need three columns from the dataset
data = df_review[['user_id', 'business_id', 'rating']]
# Instantiate a 'Reader' to read in the data so Surprise can use it
reader = Reader(rating_scale=(1, 5))


# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(data, reader=reader)

# train-test split
trainset, testset = train_test_split(data, test_size=.2)
trainset

'''The testing subset is for building your model. 
The testing subset is for using the model on unknown data to evaluate the performance 
of the model.'''

'The testing subset is for building your model. \nThe testing subset is for using the model on unknown data to evaluate the performance \nof the model.'

In [5]:
df_review.head()

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,rating,useful,funny,cool,text,date,year
0,0,0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6.0,1.0,0.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,2013.0
1,1,1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0.0,0.0,0.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,2017.0
2,2,2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3.0,0.0,0.0,I have to say that this office really has it t...,2016-11-09 20:09:03,2016.0
3,3,3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0.0,0.0,0.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,2018.0
4,4,4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7.0,0.0,0.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,2018.0


### Fitting and evaluating models

In [6]:
# instantiate SVD and fit the trainset
svd = SVD()
svd.fit(trainset)

In [7]:
predictions = svd.test(testset)
# RMSE: on average, in test set, how far off your predictions are , rating scale
accuracy.rmse(predictions) # smaller the better
accuracy.mae(predictions)

RMSE: 1.9479
MAE:  1.2855


1.2855410000590795

In [8]:
# final = SVD(n_epochs=20, n_factors=1, biased=True, 
#               lr_all=0.005, reg_all=0.06)

In [9]:
predictions[:4]

[Prediction(uid='33v8c5ftrKuDGTw1ZQ2toQ', iid='AAekuF59Qr84gAh18o9njg', r_ui=4.0, est=5, details={'was_impossible': False}),
 Prediction(uid='C349hhMXsGVdxk6u-xf6GQ', iid='bfIizwyT3-QXrntWbBpfKA', r_ui=4.0, est=5, details={'was_impossible': False}),
 Prediction(uid='mSRatm2u2bWzEbMM363PfA', iid='KsyrvUIrs6z3_c4HrA96yA', r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid='5QVF4JtSKGdr7RuwwXxQUA', iid='sdfmHavBrOZMMsiBv2sg7g', r_ui=2.0, est=5, details={'was_impossible': False})]

In [10]:
'''grid search
SVD: cold start problem : 
        solution - cosine similarity: find smoe info, to make start not cold, 
        ex. - new users: nothing to fill up
prob: popularity metrics
    
model 2: 
    2 layered content based,
    lightFM
    
content based recommendation: 
    probs: 
        
think about inputs and outputs in presentation/fair
input: drop down menu (part of dash) -  a list, dont do a df

cluster businesses, would be easier if do one type of business
'''

'grid search\nSVD: cold start problem : \n        solution - cosine similarity: find smoe info, to make start not cold, \n        ex. - new users: nothing to fill up\nprob: popularity metrics\n    \nmodel 2: \n    2 layered content based,\n    lightFM\n    \ncontent based recommendation: \n    probs: \n        \nthink about inputs and outputs in presentation/fair\ninput: drop down menu (part of dash) -  a list, dont do a df\n\ncluster businesses, would be easier if do one type of business\n'

### Gridsearch

In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier


In [12]:
from surprise.model_selection import RandomizedSearchCV

params = {'n_factors': [1,5], 'reg_all': [.005,.01,.02,.05,.1,.25,.5], 'lr_all':[.005,.01,.02,.05,.1,.25,.5],
         'n_epochs': [10,20,50]}
model = RandomizedSearchCV(SVD,param_distributions=params,n_jobs=-1,joblib_verbose=10, cv=5)
model.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/opt/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 80.9min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 133.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 198.6min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 261.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 331.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 384.9min finished


In [13]:
# best params and score
print(model.best_score)
print(model.best_params)

{'rmse': nan, 'mae': nan}
{'rmse': {'n_factors': 5, 'reg_all': 0.01, 'lr_all': 0.02, 'n_epochs': 20}, 'mae': {'n_factors': 5, 'reg_all': 0.01, 'lr_all': 0.02, 'n_epochs': 20}}


In [14]:
better = SVD(n_factors=5,reg_all=.01,lr_all=.02,n_epochs=20)
better.fit(trainset)
better_preds = better.test(testset)
accuracy.rmse(better_preds)
accuracy.mae(better_preds)

RMSE: 1.9479
MAE:  1.2855


1.2855410000590795